# Import Libraries

In [210]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import f1_score
from sklearn import metrics

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from imblearn.over_sampling import SMOTE

# Bring in Data

In [211]:
train = pd.read_csv('../input/climate-change-edsa2020-21/train.csv')
test = pd.read_csv('../input/climate-change-edsa2020-21/test.csv')

# Data pre-processing

In [212]:

## Remove urls
print ('Removing URLs...')
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
#train['message'] = train['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)
#test['message'] = test['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)

# Make lower case
print ('Lowering case...')
#train['message'] = train['message'].str.lower()
#test['message'] = test['message'].str.lower()

# Remove punctuation
import string
#print ('Cleaning punctuation...')
def remove_punctuation_numbers(post):
    punc_numbers = string.punctuation + '0123456789'
    return ''.join([l for l in post if l not in punc_numbers])
#train['message'] = train['message'].apply(remove_punctuation_numbers)
#test['message'] = test['message'].apply(remove_punctuation_numbers)



Removing URLs...
Lowering case...


In [213]:
# initialise common text & social media abbreviations as dictionary
text_abb = {"aka": "also known as", "btw": "by the way",
            "b/c": "because", "fyi": "for your information",
            "idk": "i do not know", "lol": "laughing out loud",
            "lmao": "laughing my ass off", "lmfao": "laughing",
            "omg": "oh my god", "otoh": "on the other hand",
            "wth": "what the hell", "wtf": "what the fuck",
            "icymi": "in case you missed it",
            "rofl": "rolling on the floor laughing",
            "stfu": "shut the fuck up", "nvm": "nevermind",
            "luv": "love", "luvs": "loves","qanda":"question and answer",
            "maga": "make america great again"
            }
# update text abbreviations
text_abb['nodapl'] = 'no dakota access pipeline'

In [214]:
# create stop words list
all_stopwords = stopwords.words('english')
# update stop words list for specific project
all_stopwords.extend(['amp']) 
all_stopwords.remove('not')

In [215]:
# initialise common contractions as dictionary
contractions = {"ain't": "is not", "aren't": "are not", "can't": "cannot",
                "can't've": "cannot have", "'cause": "because",
                "could've": "could have", "couldn't": "could not",
                "couldn't've": "could not have", "didn't": "did not",
                "doesn't": "does not", "don't": "do not",
                "everyone's": "everyone is", "finna": "going to",
                "gimme": "give me", "gonna": "going to", "gotta": "got to",
                "hadn't": "had not", "hadn't've": "had not have",
                "hasn't": "has not", "haven't": "have not", "he'd": "he would",
                "he'd've": "he would have", "he'll": "he will",
                "he'll've": "he he will have", "he's": "he is",
                "how'd": "how did", "howdy": "how do you do",
                "how'll": "how will", "how's": "how is", "i'd": "i would",
                "i'd've": "i would have", "i'll": "i will",
                "i'll've": "i will have", "i'm": "i am",
                "i'm'a": "I am about to", "i've": "i have",
                "innit": "is it not", "isn't": "is not", "it'd": "it would",
                "it'll": "it will", "it'll've": "it will have",
                "it's": "it is", "let's": "let us", "ma'am": "madam",
                "mayn't": "may not", "might've": "might have",
                "mightn't": "might not", "mightn't've": "might not have",
                "must've": "must have", "mustn't": "must not",
                "mustn't've": "must not have", "needn't": "need not",
                "needn't've": "need not have",
                "o'clock": "of the clock", "oughtn't": "ought not",
                "oughtn't've": "ought not have", "shan't": "shall not",
                "sha'n't": "shall not", "shan't've": "shall not have",
                "she'd": "she would", "she'd've": "she would have",
                "she'll": "she will", "she'll've": "she will have",
                "she's": "she is", "should've": "should have",
                "shouldn't": "should not", "shouldn't've": "should not have",
                "so've": "so have", "so's": "so as", "that'd": "that would",
                "that'd've": "that would have", "that's": "that is",
                "there'd": "there would", "there'd've": "there would have",
                "there's": "there is", "they'd": "they would",
                "they'd've": "they would have", "they'll": "they will",
                "they'll've": "they will have", "they're": "they are",
                "they've": "they have", "to've": "to have",
                "wasn't": "was not", "we'd": "we would",
                "we'd've": "we would have", "we'll": "we will",
                "we'll've": "we will have", "we're": "we are",
                "we've": "we have", "weren't": "were not",
                "what'll": "what will", "what'll've": "what will have",
                "what're": "what are", "what's": "what is",
                "what've": "what have", "when's": "when is",
                "when've": "when have", "where'd": "where did",
                "where's": "where is", "where've": "where have",
                "who'll": "who will", "who'll've": "who will have",
                "who's": "who is", "who've": "who have", "why's": "why is",
                "why've": "why have", "will've": "will have",
                "won't": "will not", "won't've": "will not have",
                "would've": "would have", "wouldn't": "would not",
                "wouldn't've": "would not have",
                "y'all": "you all", "y'all'd": "you all would",
                "y'all'd've": "you all would have", "y'all're": "you all are",
                "y'all've": "you all have", "you'd": "you would",
                "you'd've": "you would have", "you'll": "you will",
                "you'll've": "you will have", "you're": "you are",
                "you've": "you have"
                }

In [216]:
def tweet_preprocessing(df, cont_dict, abbr_dict, all_stopwords):
    """
    This function conducts preprocessing of tweet data for a
    machine learning model including lowercasing, expansion of
    contractions & common text abbrieviations, and removal of
    punctuation and stopwords and stemming.
    
    Arguments:
    df: original dataframe column containing tweets
    cont_dict: dictionary of contractions to be expanded
    abbr_dict: dictionary of text abbreviations to be expanded
    all_stopwords: list of stopwords to be removed
    
    Returns:
    modified dataframe of cleaned tweets
    """
    # convert to lower case
    clean_df = df.str.lower()
    # expand contractions & abbreviations
    clean_df = clean_df.replace(contractions, regex=True)
    clean_df = clean_df.replace(text_abb, regex=True)
    # replace urls
    pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
    clean_df = clean_df.str.replace(pattern_url, "weburl", regex=True)
    # remove punctuation & noise
    clean_df = clean_df.str.replace("[^a-z@#!?]", " ")
    clean_df = clean_df.apply(lambda x: ' '.join([word for word in x.split()
                                                  if len(word) > 2]))
    # remove stop words
    clean_df = clean_df.apply(lambda x: ' '.join
                             ([word for word in x.split()
                               if word not in set(all_stopwords)]))
    # reduce word to root form using lemmatization
    tweet_corpus = []
    for i in range(0, len(df)):
        tweet = clean_df[i]
        lemmatizer = WordNetLemmatizer()
        tweet = [lemmatizer.lemmatize(word) for word in tweet.split()]
        tweet = ' '.join(tweet)
        tweet_corpus.append(tweet)
    return clean_df

In [217]:
# process the train and test tweets
#train['message'] = tweet_preprocessing(train['message'], contractions,text_abb, all_stopwords)
#test['message'] = tweet_preprocessing(test['message'], contractions,text_abb, all_stopwords)

# Balance the data

In [218]:
# create the  object with the desired sampling strategy.
#smote = SMOTE(sampling_strategy='minority')

# fit object to training data
#X_train_smote, y_train_smote = smote.fit_sample(X, y)
#X_train_smote.shape

In [219]:
from sklearn.utils import resample
#Separate minority and majority classes
believe = train[train['sentiment'] == 1]
no_belief = train[train['sentiment'] == -1]
neutral = train[train['sentiment'] == 0]
news = train[train['sentiment'] == 2]

#Upsample minority class
no_belief_upsampled= resample(no_belief,replace= True,
                            n_samples= len(believe), random_state =0) #sample with replacement
neutral_upsampled= resample(neutral,replace= True,
                            n_samples= len(believe), random_state =0) #sample with replacement
news_upsampled= resample(news,replace= True,
                            n_samples= len(believe), random_state =0) #sample with replacement


#Combine majority class with upsampled minority class
df_upsampled = pd.concat ([believe,no_belief_upsampled,neutral_upsampled,news_upsampled])
#Display new class counts
df_upsampled.sentiment.value_counts()

-1    8530
 2    8530
 1    8530
 0    8530
Name: sentiment, dtype: int64

# Split

In [220]:
# If using resampling
#y = df_upsampled['sentiment']
#X = df_upsampled['message']

# No resampling
y = train['sentiment']
X = train['message']

# Vectorize the data that model requires

In [221]:
vectorizer = TfidfVectorizer(min_df=2, max_df=0.9, 
                      ngram_range=(1, 2), use_idf=2, smooth_idf=2, analyzer='word',
                      sublinear_tf = True)
X_vectorized = vectorizer.fit_transform(X)

In [222]:
X_vectorized.shape

(15819, 33271)

# Splitting the training data into a training and validation set

In [223]:
X_train,X_test,y_train,y_test = train_test_split(X_vectorized,y,test_size=0.1, stratify=y, shuffle=True,random_state=0)


# Training the model 

In [224]:
# Linear SVC model 
#lin_svc = LinearSVC()
lin_svc = LinearSVC(C=0.7)
lin_svc.fit(X_train, y_train)
lin_svc_pred_train = lin_svc.predict(X_train)   
lin_svc_pred_test = lin_svc.predict(X_test)

# Evaluation and performance

In [225]:
# Linear SVC
print(f1_score(y_train, lin_svc_pred_train, average="macro"))
print(f1_score(y_test, lin_svc_pred_test, average="macro"))

0.9807518218398193
0.7024633578008339


In [226]:
# Linear SVC
print(metrics.confusion_matrix(y_train, lin_svc_pred_train))
print(metrics.classification_report(y_test, lin_svc_pred_test))

[[1131    4   27    4]
 [   1 2010   88   19]
 [   0    9 7619   49]
 [   0    0   46 3230]]
              precision    recall  f1-score   support

          -1       0.78      0.52      0.63       130
           0       0.69      0.44      0.54       235
           1       0.79      0.90      0.84       853
           2       0.79      0.81      0.80       364

    accuracy                           0.78      1582
   macro avg       0.76      0.67      0.70      1582
weighted avg       0.78      0.78      0.77      1582



# Prepare for submission

In [227]:
testx = test['message']
test_vect = vectorizer.transform(testx)

In [228]:
y_pred_actual = lin_svc.predict(test_vect)

In [229]:
submission = pd.DataFrame({'tweetid': test['tweetid'],'sentiment': y_pred_actual})
submission.to_csv('submission.csv', index=False)
submission.head()

,tweetid,sentiment
0,169760,1
1,35326,1
2,224985,1
3,476263,1
4,872928,0
